In [ ]:
print('setup working!')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('healthcare_messy_data.csv')
df_clean = df.copy()
print(df_clean.head())

In [ ]:
df_clean.shape
df_clean.info()
df_clean.columns

In [ ]:
df_clean.describe()

In [ ]:
#Data Cleaning

In [ ]:
df_clean.columns = df_clean.columns.str.strip()

In [ ]:
#remove white spaces from column names
df_clean.columns = df_clean.columns.str.strip().str.lower().str.replace(' ', '_')

In [ ]:
df_clean.columns

In [ ]:
#check missing data 
missing = df_clean.isnull().sum().sort_values(ascending=False)
missing_percentage = (df_clean.isnull().sum()/df_clean.isnull().count()*100).sort_values(ascending=False)
pd.concat([missing, missing_percentage], axis=1, keys=['Total Missing', 'Percentage Missing']).transpose()   


In [ ]:
#age

In [ ]:
#check unique in age column
df_clean['age'].unique()

In [ ]:
#fix age column
#change forty to 40
df_clean['age'] = df_clean['age'].replace('forty', '40')
# convert age column to numeric values, forcing errors to NaN
df_clean['age'] = pd.to_numeric(df_clean['age'], errors='coerce')
# fill missing or NaN values with the median age
df_clean['age'] = df_clean['age'].fillna(df_clean['age'].median())
#change data type to integer
df_clean['age'] = df_clean['age'].astype(int)


In [ ]:
df_clean['age'].head()

In [ ]:
# check cholesterol column
df_clean['cholesterol'].unique()

In [ ]:
#fix cholesterol column
# correct non-numeric values in cholesterol column
df_clean['cholesterol'] = pd.to_numeric(df_clean['cholesterol'], errors='coerce')
# fill missing or NaN values with the median cholesterol
df_clean['cholesterol'] = df_clean['cholesterol'].fillna(df_clean['cholesterol'].median())
#change data type to integer
df_clean['cholesterol'] = df_clean['cholesterol'].astype(int)

In [ ]:
df_clean['cholesterol'].head()

In [ ]:
#blood pressure

In [ ]:
#check blood pressure column
df_clean['blood_pressure'].unique()

In [ ]:
#fix blood pressure column
#split the blood pressure column into systolic and diastolic
bp_split = df_clean['blood_pressure'].str.split('/', expand=True)
#assign systolic and diastolic to new columns
df_clean['systolic'] = pd.to_numeric(bp_split[0], errors='coerce')
df_clean['diastolic'] = pd.to_numeric(bp_split[1], errors='coerce')
#fill missing values with median
df_clean['systolic'] = df_clean['systolic'].fillna(df_clean['systolic'].median())
df_clean['diastolic'] = df_clean['diastolic'].fillna(df_clean['diastolic'].median())
#change data type to integer
df_clean['systolic'] = df_clean['systolic'].astype(int)
df_clean['diastolic'] = df_clean['diastolic'].astype(int)
# recreate blood pressure column
# Extract systolic BP as float
df_clean['blood_pressure'] = df_clean['systolic'].astype(str) + '/' + df_clean['diastolic'].astype(str)
                                                     
                                     

In [ ]:
#check blood pressure column again
df_clean[['blood_pressure','systolic','diastolic']].head()

In [ ]:
#condition and mediction

In [ ]:
#check condition column
df_clean['condition'].unique()

In [ ]:
#fix condition column 
#fill nan missing value with unknown
df_clean['condition'] = df_clean['condition'].fillna('Unknown')
#change data type to catagory
df_clean['condition'] = df_clean['condition'].astype('category')


In [ ]:
#check medication column
df_clean['medication'].unique()

In [ ]:
#fix medication column
#change data type
df_clean['medication'] = df_clean['medication'].astype('category')

In [ ]:
df_clean['medication'].dtypes

In [ ]:
# define correct condition-to-medication mapping
correct_mapping = {
    'Diabetes': 'METFORMIN',
    'Heart Disease': 'ATORVASTATIN',
    'Hypertension': 'LISINOPRIL',
    'Asthma': 'ALBUTEROL',
    None: 'NONE'  # for 'nan' or None condition, set medication to 'NONE'
}

In [ ]:
# function to clean and validate medication based on condition
def clean_medication(row):
    condition = row['condition']
    # Check if the medication matches the correct mapping for the condition
    if row['medication'] != correct_mapping.get(condition, None):
        return correct_mapping.get(condition, 'NONE')  # if mismatch, replace with the correct medication
    return row['medication']  # if it's correct, keep the same

In [ ]:
# apply the function to the dataset
df_clean['medication'] = df_clean.apply(clean_medication, axis=1) 

In [ ]:
df_clean[['medication','condition']].head()

In [ ]:
#visit date

In [ ]:
#check visit date
df_clean['visit_date'].unique()

In [ ]:
# function to standardize dates into a consistent `YYYY-MM-DD` format
def standardize_date(date):
    formats = ['%m/%d/%Y', '%B %d, %Y', '%Y.%m.%d', '%Y/%m/%d', '%m-%d-%Y']
    for fmt in formats:
        try:
            return pd.to_datetime(date, format=fmt)
        except ValueError:
            continue
    return pd.NaT  # if all formats fail, return NaT

# apply the function to the column
df_clean['visit_date'] = df_clean['visit_date'].apply(standardize_date)

# format all valid dates to 'YYYY-MM-DD'
df_clean['visit_date'] = df_clean['visit_date'].dt.strftime('%Y-%m-%d')



In [ ]:
df_clean['visit_date'] = pd.to_datetime(
    df_clean['visit_date'],
    errors='coerce',
    infer_datetime_format=True
)

In [ ]:
#check visit date data type
df_clean['visit_date'].dtypes

In [ ]:
print(df_clean.columns.tolist())


In [ ]:
df_clean['visit_date'].head()

In [ ]:
#gender

In [ ]:
#check gender column
df_clean['gender'].unique()

In [ ]:
#fix gender colum
#fix data type
df_clean['gender'] = df_clean['gender'].astype('category')

In [ ]:
#patient name

In [ ]:
#generate patient id for private use
df_clean['patient_id'] = ['P' + str(i).zfill(5) for i in range(1, len(df_clean)+1)]
df_clean = df_clean[['patient_id'] + [c for c in df_clean.columns if c != 'patient_id']]


In [ ]:
#drop patient name, email and number columns for privacy
df_clean = df_clean.drop(columns=['patient_name', 'email', 'phone_number'],  errors='ignore')

In [ ]:
df_clean.head(10)

In [ ]:
df_clean.dtypes

In [ ]:
#filter

In [ ]:
# first categorise to make things easier for eda

In [ ]:
# group age into categories
def age_group(age):
  if age <=18:
    return 'Child'
  elif age <=35:
    return 'Young '
  elif age <=60:
    return 'Adult'
  else:
    return 'Senior'

df_clean['age_group'] = df_clean['age'].apply(age_group)

In [ ]:
#blood pressure levels
def bp_category(row):
    sys = row['systolic']
    dia = row['diastolic']

    if sys < 120 and dia < 80:
        return 'Normal'
    elif 120 <= sys < 130 and dia < 80:
        return 'Elevated'
    elif sys >= 130 or dia >= 80:
        return 'High'
    else:
        return 'Unknown'

df_clean['bp_level'] = df_clean.apply(bp_category, axis=1)


In [ ]:
# cholesterol levels
def cholesterol_level(value):
    if value < 200:
        return 'Normal'
    elif 200 <= value < 240:
        return 'Borderline High'
    else:
        return 'High'

df_clean['cholesterol_level'] = df_clean['cholesterol'].apply(cholesterol_level)


In [ ]:
#categorize who is on medication
df_clean['on_medication'] = df_clean['medication'].apply(lambda x: 'Yes' if x != 'NONE' else 'No')


In [ ]:
print(df_clean.head(10))

In [ ]:
df_clean.head(10)

In [ ]:
#change datatypes for bp level , cholesterol level , age group and on medication
df_clean['age_group'] = df_clean['age_group'].astype('category')
df_clean['bp_level']= df_clean['bp_level'].astype('category')
df_clean['cholesterol_level']= df_clean['cholesterol_level'].astype('category')
df_clean['on_medication']= df_clean['on_medication'].astype('bool')




In [ ]:
# eda

In [ ]:
#overall age  distribution
figs = plt.subplots(1, 1, figsize=(10, 6))
sns.histplot(data=df_clean, x='age', bins=30, kde=True, color='skyblue')
plt.title('Overall Age Distribution')
xticks = plt.xticks()
plt.show()

In [ ]:
#Most patients are in their mid-30s to 40s, with fewer youth and seniors, making the dataset skewed toward working adults. This reflects adult living patterns while underrepresenting younger and older citizens.

In [ ]:
#age group
figs = plt.subplots(1, 1, figsize=(10, 6))
colors = sns.color_palette('pastel')
plt.pie(df_clean['age_group'].value_counts(), labels=df_clean['age_group'].value_counts().index, colors=colors, autopct='%1.1f%%')
plt.title('Age Group')
plt.show()

In [ ]:
#Adults dominate the dataset, followed by youth and then seniors, confirming an imbalance that is common in health data.

In [ ]:
#gender distribution
figs = plt.subplots(1, 1, figsize=(8, 5))
colors = sns.color_palette('Set1')
plt.pie(df_clean['gender'].value_counts(), labels=df_clean['gender'].value_counts().index, colors=colors, autopct='%1.1f%%')
plt.title('Gender Distribution')
plt.show()

In [ ]:
#Gender distribution appears even, but the “Other” category may represent non-binary individuals or data collection issues. If misclassified, this variable could lead to misleading results.

In [ ]:
# 3. Combined risk: High BP AND High Cholesterol 
df_clean['high_risk'] = ( (df_clean['bp_level'] == 'High') & (df_clean['cholesterol_level'] == 'High') )
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
sns.countplot(data=df_clean, x='age_group', hue='high_risk', palette='Set1', ax=axes[0])
axes[0].set_title('Age Group vs Combined High BP & Cholesterol')
sns.countplot(data=df_clean, x='gender', hue='high_risk', palette='Set1', ax=axes[1])
axes[1].set_title('Gender vs Combined High BP & Cholesterol')
for ax in axes:
    total = len(df_clean)
    for p in ax.patches:
        height = p.get_height()
        ax.annotate(f'{100*height/total:.1f}%', 
                    (p.get_x() + p.get_width()/2., height), 
                    ha='center', va='bottom')
plt.show()

In [ ]:
#Adults show the highest rates of combined high blood pressure and cholesterol, linked to stressful jobs and unhealthy habits. Seniors, though biologically more vulnerable, are underrepresented, while 33.4% of young adults being affected is concerning. Gender distribution is nearly equal, showing health risks here are not strongly gender-driven.

In [ ]:
#Age Group vs Blood Pressure & Cholesterol Levels
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Age group vs blood pressure

sns.boxplot(data=df_clean, x='age_group', y='systolic', palette='Set3', ax=axes[0])
axes[0].set_title('Age Group vs. Systolic Blood Pressure')
# Age group vs cholesterol
sns.boxplot(data=df_clean, x='age_group', y='cholesterol', palette='Set1', ax=axes[1])
axes[1].set_title('Age Group vs. Cholesterol Levels')

plt.show()


In [ ]:
#Systolic blood pressure values are consistent for youth and seniors, while adults show more variability due to their larger dataset share. Cholesterol levels are also consistent with no extreme outliers, suggesting most individuals are relatively healthy, though this contrasts with real-world trends where age often plays a role.

In [ ]:
# Gender vs Cholesterol Levels 
fig, axes = plt.subplots(1, 2, figsize=(16, 6)) 
sns.boxplot(data=df_clean, x='gender', y='systolic', palette='Set1', ax=axes[0])
axes[0].set_title('Gender vs. Cholesterol')
sns.boxplot(data=df_clean, x='gender', y='cholesterol', palette='Set1', ax=axes[1])
axes[1].set_title('Gender vs. Cholesterol Levels')
plt.show()

In [ ]:
#Gender differences in cholesterol and blood pressure are minimal, with short box plots and no extreme outliers. This suggests an evenly skewed dataset, which is unusual compared to real-world findings where gender often influences risk.

In [ ]:

# Ddistribution based on condition
fig, ax = plt.subplots(figsize=(10, 6))
sns.countplot(data=df_clean, x='condition', palette='Set2', ax=ax)
ax.set_title('Condition Distribution')
total = len(df_clean)
for p in ax.patches:
    height = p.get_height()
    ax.annotate(f'{100*height/total:.1f}%', 
                (p.get_x() + p.get_width()/2., height), 
                ha='center', va='bottom')

plt.show()


In [ ]:
#Conditions are almost equally distributed, with asthma slightly leading at 21.1%. Most individuals live with chronic illnesses and are medicated, while the “Unknown” group may represent undiagnosed cases. This highlights the need for interventions beyond blood pressure and cholesterol alone.

In [ ]:
#age group vs condition and gender vs condition 
figs = plt.subplots(1, 2, figsize=(14, 8))
sns.countplot(data=df_clean, x='age_group', hue='condition', palette='Set2', ax=figs[1][0])
axes[0].set_title('Age Group vs. Condition')
sns.countplot(data=df_clean, x='gender', hue='condition', palette='Set2', ax=figs[1][1])
axes[1].set_title('Gender vs. Condition')
plt.show()

In [ ]:
#Adults are most affected, especially by diabetes, reflecting unhealthy lifestyle habits like high sugar intake and energy drinks. Youth show more asthma cases, pointing to genetic or environmental factors, while seniors are vulnerable across all diseases. Gender differences exist: women lean toward asthma, men toward diabetes, and others toward heart disease.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Age group vs medication
sns.countplot(data=df_clean, x='age_group', hue='on_medication', palette='Set1', ax=axes[0])
axes[0].set_title('Age Group vs. On Medication')

# Gender vs medication
sns.countplot(data=df_clean, x='gender', hue='on_medication', palette='Set1', ax=axes[1])
axes[1].set_title('Gender vs. On Medication')

# Add percentage labels
for ax in axes:
    total = len(df_clean)
    for p in ax.patches:
        height = p.get_height()
        ax.annotate(f'{100*height/total:.1f}%', 
                    (p.get_x() + p.get_width()/2., height), 
                    ha='center', va='bottom')

plt.show()


In [ ]:
#Adults are the most medicated, reflecting their higher chronic disease burden. Youth medication use is worrying, showing illnesses start earlier, while seniors being less medicated highlights possible gaps in healthcare access. Gender differences are negligible, showing medication use is not strongly gender-driven.

In [ ]:
df_clean['visit_date'] = pd.to_datetime(df_clean['visit_date'])

df_clean['visit_year'] = df_clean['visit_date'].dt.year
df_clean['visit_month'] = df_clean['visit_date'].dt.month_name()
df_clean['visit_month_num'] = df_clean['visit_date'].dt.month

import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Visits per Year
year_counts = df_clean['visit_year'].value_counts().sort_index()

sns.barplot(
    x=year_counts.index,
    y=year_counts.values,
    ax=axes[0]
)

axes[0].set_title("Number of Patient Visits per Year")
axes[0].set_xlabel("Year")
axes[0].set_ylabel("Number of Visits")


#  Visits per Month
month_counts = (
    df_clean
    .groupby(['visit_month_num', 'visit_month'])
    .size()
    .reset_index(name='visits')
    .sort_values('visit_month_num')
)

sns.barplot(
    data=month_counts,
    x='visit_month',
    y='visits',
    ax=axes[1]
)

axes[1].set_title("Number of Patient Visits per Month")
axes[1].set_xlabel("Month")
axes[1].set_ylabel("Number of Visits")

plt.tight_layout()
plt.show()


In [ ]:
#Visits peaked in 2019–2020, reflecting heightened health awareness during the pandemic. The steady monthly average of four visits suggests routine management of chronic conditions rather than crisis-driven care.

In [ ]:
#Overall, the dataset highlights adult lifestyle-driven risks, underrepresentation of seniors, and balanced gender effects

In [ ]:
df_clean.head()

In [ ]:
#delete unnecessary columns
df_clean=df_clean.drop(columns=['on_medication', 'visit_month', 'visit_year','high_risk','visit_month_num'],  errors='ignore')

In [ ]:
df_clean.head()

In [ ]:
#save into csv
df_clean.to_csv('healthcare_clean_data.csv', index=False)